<a href="https://colab.research.google.com/github/russodanielp/intro_cheminformatics/blob/google_colab/Lab%20XX%20-%20Cheminformatics%20Tools/QSAR-Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QSAR - Classification

Build either a Random Forest or k-Nearest Neighbor classification model using scikit-learn.  

## Script variables

Script variables that need to be changed are in the script below.  The script requires that you provide four pieces of information.  

1) `SDFILE_DIR`: the filepath to the SDFile containing the chemicals to build a ML model  
2) `ACTIVITY_COLUMN`: the name of the column/property in the SDFile that contains the activity you would like to perform QSAR on  
3) `NAME_COLUMN`: the name of the column/property in the SDFile that contains the name or identifier of the molecule  
4) `ALGORITHM`: which machine learning algorithm you would like to use - either random forest (rf) or k-nearest neighbors (knn)  
5) `FEATURES`: which set of features to represent chemicals (ECFP6, FCFP6, MACCS)

In [ ]:
!pip install rdkit

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 37.8 MB/s eta 0:00:00


In [ ]:
SDFILE_DIR = 'Training.sdf'
ACTIVITY_COLUMN = 'Composite category'
NAME_COLUMN = 'CASRN '
ALGORITHM = 'rf'
FEATURES = 'ecfp6'

First, import the necessary packages.

In [ ]:
# Standard imports
import pandas as pd
import numpy as np
import os
from rdkit import Chem 
from rdkit.Chem import PandasTools
from rdkit.ML.Descriptors import MoleculeDescriptors
from rdkit.Chem import AllChem
from rdkit.Chem import Descriptors
from rdkit.Chem import AllChem
from rdkit.Chem import MACCSkeys
from rdkit import Chem
import pandas as pd

# ML imports
from sklearn import model_selection
from sklearn import pipeline
from sklearn.model_selection import cross_val_predict
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from sklearn.metrics import mean_absolute_error, r2_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

# Plotting imports
import seaborn as sns
import matplotlib.pyplot as plt

## Data preprocessing

Here we create desctipors for the chemicals and prepare the descriptor matrix and the and the activity column we want to learn.

In [ ]:
def calc_fp_from_mol(mol, method="maccs", n_bits=2048):
    """
    Encode a molecule from a RDKit Mol into a fingerprint.

    Parameters
    ----------
    mol : RDKit Mol
        The RDKit molecule.

    method : str
        The type of fingerprint to use. Default is MACCS keys.

    n_bits : int
        The length of the fingerprint.

    Returns
    -------
    array
        The fingerprint array.

    """
    method = method.lower()
    if method == "maccs":
        return list(MACCSkeys.GenMACCSKeys(mol))
    elif method == "ecfp4":
        return list(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=n_bits))
    elif method == "ecfp6":
        return list(AllChem.GetMorganFingerprintAsBitVect(mol, 3, nBits=n_bits))
    else:
        print(f"Warning: Wrong method specified: {method}. Default will be used instead.")
        return list(MACCSkeys.GenMACCSKeys(mol))

Loop through the data frame and create chemical descriptors for each molecule. 

In [ ]:
df = PandasTools.LoadSDF(SDFILE_DIR)

desc_list = []

for mol in df.ROMol.tolist():
    desc = calc_fp_from_mol(mol, method=FEATURES)
    desc_list.append(desc)
    
desc_frame = pd.DataFrame(desc_list)

Create variables to feed into the ML algorithm.  `X` is the descriptor matrix and `y` contains the activity to predict.

In [ ]:
X = desc_frame.copy()
X.index = df[NAME_COLUMN].astype(str)
y = df[ACTIVITY_COLUMN].astype(float)
y.index = df[NAME_COLUMN].astype(str)

print(X.shape)
print(y.shape)

(75, 2048)
(75,)


Sometimes chemicals can not get descriptors made for them, so we need to remove these

In [ ]:
X_train = X[X.notnull().all(1)]
y_train = y[X.notnull().all(1)]

print(X_train.shape)
print(y_train.shape)

(75, 2048)
(75,)


## Model training

The model is trained using 5-fold cross validation.  Model parameters are searching using a "grid search" method that searches through all possible parameters and finds the optimal solution.  

The 5-fold cross validation predictions are exported to a file `data/five_fold_predictions.csv`.

In [ ]:
# RF
model_RF = RandomForestClassifier()

#knn
model_KNN = KNeighborsClassifier(metric='jaccard')


models = {
    "rf": model_RF,
    "knn": model_KNN
}

params_dic = {
    'rf' : {'rf__n_estimators': [10, 25, 50, 100]},
    'knn': {'knn__n_neighbors': [1, 2, 3, 4, 5, 10, 25]}
}

In [ ]:
N_FOLDS = 5

clf = models[ALGORITHM]
params = params_dic[ALGORITHM]
pipe = pipeline.Pipeline([(ALGORITHM, clf)])
cv = KFold(n_splits=N_FOLDS, shuffle=True, random_state=0)
grid_search = model_selection.GridSearchCV(pipe, param_grid=params, cv=cv, refit='AUC')
grid_search.fit(X_train, y_train)
best_estimator = grid_search.best_estimator_

Export the predictions

In [ ]:
preds = pd.DataFrame(cross_val_predict(best_estimator, X_train, y_train), 
                    index=X_train.index, columns=['CV Prediction'])
preds['CV Prediction'] = preds['CV Prediction'].round(2)
preds['Compound'] = X_train.index
preds[['Compound', 'CV Prediction']].to_csv('five_fold_predictions.csv', index=False)

## Test Set Predictions

In [ ]:
TEST_SDFILE_DIR = 'External.sdf'
TEST_NAME_COLUMN = 'CASRN '

In [ ]:
test_set = PandasTools.LoadSDF(TEST_SDFILE_DIR)

desc_list = []

for mol in test_set.ROMol.tolist():
    desc = calc_fp_from_mol(mol, method=FEATURES)
    desc_list.append(desc)
    
X_test = pd.DataFrame(desc_list)

In [ ]:
preds = pd.DataFrame(best_estimator.predict(X_test), 
                    index=X_test.index, columns=['Prediction'])
preds['Prediction'] = preds['Prediction'].round(2)
preds['Compound'] = test_set[TEST_NAME_COLUMN]
preds[['Compound', 'Prediction']].to_csv('test_set_predictions.csv', index=False)